### Mapping variable IDs...

# TODO: 
* make this so it can run SQL queries directly to ICCA - to get attributes
* add config file that defined name or server and database (and table names?). instruct user to set these first.
* add instructions for setup and running streamlit app. (maybe remove jupyerlab?)
* write a python script that will run all the initial SQL queries (to get intervention that are in use in each table) and save them to `data/` directory.

#### The procedure is:
- run script(s) to produce complete intervention tables for each db table (e.g. PtAssessment)
- use search strings to query those tables for each variable of interest, selecting relevant interventionIds manually
- for each interventionId requested, run a query to produce example data with each attribute for that intervention
- review outputs manually and select intervention/attribute pairs, giving a name to each and adding any comments as free text
- save these in a suitable formate (one that allows to check for completeness)

#### Notes:

- In some cases there may be many unrelated or irrelevant interventions pciked up by our search strings. This is the price we pay for trying tn esnure that our search is not missing anything!

In [26]:
import pandas as pd
import pyodbc
from functools import reduce
import numpy as np

In [2]:
schema = pd.read_excel(
    '../schema/smartt_variable_definitions.xlsx', 
    sheet_name='search_strings'
)

In [12]:
schema.head(45)

,Variable,Simple name,Search Strings,Likely ICCA table,Possibly derived,Pacmed ontology,Snowmed CT,Notes
0,Age at admission,age,age,PtDemographics,True,General_information.Patient_characteristics,NaN,NaN
1,Gender,gender,gender,PtDemographics,False,General_information.Patient_characteristics,NaN,NaN
2,Weight at admission,weight,weight,PtDemographics,True,General_information.Patient_characteristics,NaN,NaN
3,Origin department,admit_from,"admit, origin, admission, location",?,True,General_information.Admission_information,NaN,NaN
4,pH,ph,ph,PtLabResults,False,Laboratory_results.Blood_gas_analysis,NaN,NaN
5,Paco2,paco2,"paco2, co2, carbon dioxide",PtLabResults,False,Laboratory_results.Blood_gas_analysis,NaN,NaN
6,Pao2,pao2,"pao2, o2, oxygen",PtLabResults,False,Laboratory_results.Blood_gas_analysis,NaN,NaN
7,actual bicarbonate,hco3,"bicarbonate, hco3, bicarb, co2, carbon dioxide",PtLabResults,False,Laboratory_results.Blood_gas_analysis,NaN,NaN
8,base excess,base_excess,"base excess, base, BE",PtLabResults,False,Laboratory_results.Blood_gas_analysis,NaN,NaN
9,arterial oxygen saturation,so2,"saturation, sao2, so2, o2sat",PtLabResults,False,Laboratory_results.Blood_gas_analysis,NaN,Should we include/combine SpO2 in this variabl...


In [4]:
interventions = pd.read_csv('../data/all_ptassessment_interventions_units_5_8_9.rpt', sep='\t')
interventions.dropna(axis=0, subset=['shortLabel', 'longLabel'], inplace=True)
interventions.head()

,interventionId,shortLabel,longLabel,conceptCode,numberOfPatients,firstChartTime,lastChartTime,numberOfRecords,minClinicalUnitId,maxClinicalUnitId
1,1746,Airway Respiratory Rate,Airway Respiratory Rate,86290005.0,53345.0,2015-02-03 12:00:00.000,2016-05-30 15:00:00.000,53345.0,5.0,5.0
2,1793,RVSWI,Right Ventricular Stroke Work Index,277380003.0,875.0,2015-05-26 17:55:00.000,2023-06-08 08:00:00.000,875.0,5.0,8.0
3,1798,LVSWI,Left Ventricular Stroke Work Index,276898003.0,464.0,2015-05-26 17:55:00.000,2023-06-08 13:00:00.000,464.0,5.0,8.0
4,1830,Glasgow Coma,Glasgow Coma Scale,386554004.0,2751747.0,2015-02-03 10:00:00.000,2023-08-10 00:00:00.000,2751747.0,5.0,9.0
5,1844,LCWI,Left Cardiac Work Index,399266005.0,480.0,2015-05-26 17:55:00.000,2023-06-08 13:00:00.000,480.0,5.0,8.0


In [5]:
variables = list(schema.Variable)
print(f"There are {len(variables)} variables in the schema.")

There are 65 variables in the schema.


In [13]:
variable_id = 41

In [22]:
search_strings = [
    s.strip()
    for s in 
    schema.loc[variable_id]['Search Strings'].split(',')
]

In [23]:
search_strings

['nbp', 'nibp', 'noninvasive', 'non-invasive', 'pressure', 'systolic']

In [24]:
logical_index = np.logical_or.reduce(
    [
        interventions.longLabel.str.contains(search_string, case=False)
        for search_string in search_strings
    ]
)

In [25]:
display_cols = ['interventionId', 'shortLabel', 'longLabel', 'numberOfPatients', 'firstChartTime', 'firstChartTime']
interventions[logical_index][display_cols]

,interventionId,shortLabel,longLabel,numberOfPatients,firstChartTime,firstChartTime
67,2770,PAP,Pulmonary Artery Pressure,22644.0,2015-05-26 16:00:00.000,2015-05-26 16:00:00.000
77,2899,Pressure Areas Intact?,Pressure Areas Intact?,13467.0,2015-03-24 08:37:00.000,2015-03-24 08:37:00.000
117,2946,Pre-Op Diastolic Blood Pressure,Pre-Op Diastolic Blood Pressure (mmHg):,20205.0,2015-02-27 17:25:00.000,2015-02-27 17:25:00.000
119,2948,Pre-Op Systolic Blood Pressure,Pre-Op Systolic Blood Pressure (mmHg):,20826.0,2015-02-27 17:25:00.000,2015-02-27 17:25:00.000
146,2980,Systolic BP Limit (mmHg):,Systolic BP Limit (mmHg):,6965.0,2015-03-06 13:04:00.000,2015-03-06 13:04:00.000
272,3133,Pressure Area Intact Variance,Pressure Area Intact Variance,1045.0,2015-03-24 08:37:00.000,2015-03-24 08:37:00.000
340,3218,Intra Abdominal Pressure,Intra Abdominal Pressure,602.0,2015-03-11 22:00:00.000,2015-03-11 22:00:00.000
350,3244,PS/ASB (Above PEEP),Pressure Support/ASB (Above PEEP),411985.0,2015-02-03 14:00:00.000,2015-02-03 14:00:00.000
418,3363,NBP,Non-Invasive BP,215133.0,2015-02-03 09:55:00.000,2015-02-03 09:55:00.000
420,3367,PPV,Pulse Pressure Variation,582.0,2015-10-31 00:00:00.000,2015-10-31 00:00:00.000


In [54]:
interventions.loc[145]

interventionId                          2978
shortLabel            Management Plan Other:
longLabel             Management Plan Other:
conceptCode                      305260005.0
numberOfPatients                      2872.0
firstChartTime       2015-02-27 17:25:00.000
lastChartTime        2023-08-09 16:55:00.000
numberOfRecords                       2872.0
minClinicalUnitId                        5.0
maxClinicalUnitId                        8.0
Name: 145, dtype: object

In [27]:
import sqlite3

In [51]:
con = sqlite3.connect('test.db')

In [29]:
df = pd.DataFrame({'a': 0}, index=[1,2,3])

In [33]:
df.to_sql('test', con, index=True, index_label='index')

3

In [53]:
cur = con.cursor()
cur.execute(f"""
                UPDATE test
                SET a = 3
                WHERE "index" = 3; 
            """)
con.commit()
con.close()


In [54]:
con = sqlite3.connect('test.db')
pd.read_sql("SELECT * FROM test", con)

,index,a
0,1,0
1,2,1
2,3,3
